In [80]:
import sys
import os
from datetime import datetime
import time
import pandas as pd

sys.path.append(os.path.abspath('/workspaces/projeto_ing/utils/shered_utils/'))

from sql_conn import conectar_mongodb
from sql_conn import conectar_bronze
from sql_conn import enviar_para_mongodb
from sql_conn import mensagem

''' 
Motor dedicado ao carregamento e envio de dados para o mongo.db

Utilizndo a função conectar_mongo para conexão entre fontes
Utilizndo a função conectar_bronze para realizar a transformação dos dados a serem enviados
Utilizndo a função enviar_para_mongodb para envio do DataFrame para a collection

'''
# path = '/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

def carregamento_landing(data_dict, path=None, file=None, resultado=None ):
    import os
    import sys
    
    if path is None:
        path = '/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/'
        print('caminho não informado')
        print('utilizando o padrão:')
        print(f'{path}')
    
    read_landing = os.listdir(path)
    
    for reads in read_landing:
        accept = os.path.join(path, reads)
        print(f'item encontrado: {accept}')
        
        #test para gravar conteudo de controle
        sav = os.path.basename(accept)
        print(f'Conteúdo a gravar: {sav}')
        
        backp = '/workspaces/projeto_ing/tests/app/ambiente_test_ing/backup/arquivos_mongo.txt'
        with open(backp, 'r+') as controle_ingestao: 
            conteudo_load = controle_ingestao.readlines()
            print(f'Conteúdo do arquivo de controle de ingestão: {conteudo_load}')
            
            if sav + '\n' not in conteudo_load:
                with open(backp, 'a') as controle_ingestao:
                    controle_ingestao.write(f"{sav}\n")
                    print(f'nomenclatura do arquivo salvo: {sav}')
                    print('=================================')
        
        
                if accept.endswith('.txt'):
                    resultado = pd.read_csv(accept, delimiter=';')
                    resultado = resultado.where(pd.notna(resultado), None)
                    # print(resultado)
                    data_dict = resultado.to_dict("records")
                    # mongo_collection.insert_many(data_dict)
                    print(data_dict)
                    print("Dados para inserir com sucesso no MongoDB!")

                else:
                    raise ValueError("Formato de arquivo não suportado")
            else:
                print('arquivo já inserido')

        
        

    return data_dict

In [81]:
carregamento_landing(data_dict)

caminho não informado
utilizando o padrão:
/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_teste2_20242306_215405.txt
Conteúdo a gravar: resultado_teste2_20242306_215405.txt
Conteúdo do arquivo de controle de ingestão: []
nomenclatura do arquivo salvo: resultado_teste2_20242306_215405.txt
[{'id': 1, 'nome': 'rafael', 'idade': 30}, {'id': 2, 'nome': 'joyce', 'idade': 28}]
Dados para inserir com sucesso no MongoDB!
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_cep_mairiporã_20241014_231311.txt
Conteúdo a gravar: resultado_cep_mairiporã_20241014_231311.txt
Conteúdo do arquivo de controle de ingestão: ['resultado_teste2_20242306_215405.txt\n']
nomenclatura do arquivo salvo: resultado_cep_mairiporã_20241014_231311.txt
[{'cep': '07601-345', 'endereco': 'Avenida Hum', 'bairro': "Olho D'Água", 'complemento': None, 'uf': 'SP', 'cida

[{'cep': '07601-345',
  'endereco': 'Avenida Hum',
  'bairro': "Olho D'Água",
  'complemento': None,
  'uf': 'SP',
  'cidade': 'Mairiporã',
  'estado': 'São Paulo',
  'regiao': 'Sudeste',
  'ddd': 11,
  'ibge': 3528502,
  'data': '2024-10-14'},
 {'cep': '07613-690',
  'endereco': 'Avenida Rubi',
  'bairro': 'Mantiqueira',
  'complemento': None,
  'uf': 'SP',
  'cidade': 'Mairiporã',
  'estado': 'São Paulo',
  'regiao': 'Sudeste',
  'ddd': 11,
  'ibge': 3528502,
  'data': '2024-10-14'},
 {'cep': '07613-655',
  'endereco': 'Avenida Ouro',
  'bairro': 'Mantiqueira',
  'complemento': None,
  'uf': 'SP',
  'cidade': 'Mairiporã',
  'estado': 'São Paulo',
  'regiao': 'Sudeste',
  'ddd': 11,
  'ibge': 3528502,
  'data': '2024-10-14'},
 {'cep': '07627-160',
  'endereco': 'Avenida Berna',
  'bairro': 'Luiz Fagundes',
  'complemento': '(Pq Suiço Cant 1 Gl)',
  'uf': 'SP',
  'cidade': 'Mairiporã',
  'estado': 'São Paulo',
  'regiao': 'Sudeste',
  'ddd': 11,
  'ibge': 3528502,
  'data': '2024-10-14

In [79]:
# path = '/workspaces/projeto_ing/tests/app/src/teste_landing/'
print('===============================')
client, collection = conectar_mongodb()

if collection is not None:
    carregamento_landing(data_dict, collection)


Conexão bem-sucedida com o MongoDB!
caminho não informado
utilizando o padrão:
/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_teste2_20242306_215405.txt
Conteúdo a gravar: resultado_teste2_20242306_215405.txt
Conteúdo do arquivo de controle de ingestão: []
nomenclatura do arquivo salvo: resultado_teste2_20242306_215405.txt


AttributeError: 'NoneType' object has no attribute 'insert_many'

In [60]:
client, collection = conectar_mongodb()

if collection is not None:
    enviar_para_mongodb(data_dict, collection)

Conexão bem-sucedida com o MongoDB!


TypeError: enviar_para_mongodb() missing 1 required positional argument: 'mongo_collection'

In [14]:
resultado

,cep,endereco,bairro,complemento,uf,cidade,estado,regiao,ddd,ibge,data
0,07601-345,Avenida Hum,Olho D'Água,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
1,07613-690,Avenida Rubi,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
2,07613-655,Avenida Ouro,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
3,07627-160,Avenida Berna,Luiz Fagundes,(Pq Suiço Cant 1 Gl),SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
4,07657-210,Avenida Alpes,Mato Dentro,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
...,...,...,...,...,...,...,...,...,...,...,...
166,07600-598,Rodovia Prefeito Luiz Salomao Chamma,Lavapés,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
167,07605-700,Rodovia Prefeito Luiz Salomao Chamma,Rio abaixo,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
168,07600-740,Estrada Armando Barbosa de Almeida,Chácara Arantes,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
169,07604-525,Estrada Armando Barbosa de Almeida,Caceia,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14


In [ ]:
def transf_json(consulta=None):
    
    if not consulta:
        print('Nenhum dado encontrado')
        return
    
    df = pd.DataFrame(consulta)
    return df
    
    

In [10]:
transf_json(consulta=resultado)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
def transf_jsonnn(consulta, colunas, mongo_collection):
    if not consulta:
        print("Nenhum dado para inserir no MongoDB.")
        return
    
    df = pd.DataFrame(consulta, columns=';')
    df = df.where(pd.notna(df), None)
    
    data_dict = df.to_dict("records")
    mongo_collection.insert_many(data_dict)
    print(data_dict)
    print("Dados inseridos com sucesso no MongoDB!")
    

In [3]:
print(resultado)

     07601-345                           Avenida Hum      Olho D'Água  \
0    07613-690                          Avenida Rubi      Mantiqueira   
1    07613-655                          Avenida Ouro      Mantiqueira   
2    07627-160                         Avenida Berna    Luiz Fagundes   
3    07657-210                         Avenida Alpes      Mato Dentro   
4    07613-680                        Avenida Safira      Mantiqueira   
..         ...                                   ...              ...   
165  07600-598  Rodovia Prefeito Luiz Salomao Chamma          Lavapés   
166  07605-700  Rodovia Prefeito Luiz Salomao Chamma       Rio abaixo   
167  07600-740    Estrada Armando Barbosa de Almeida  Chácara Arantes   
168  07604-525    Estrada Armando Barbosa de Almeida           Caceia   
169  07605-005    Estrada Armando Barbosa de Almeida       Rio abaixo   

               Unnamed: 3  SP  Mairiporã  São Paulo  Sudeste  11  3528502  \
0                     NaN  SP  Mairiporã  São 

In [94]:
def enviar_para_mongodb(consulta, colunas, mongo_collection):
    """
    Converte os dados do SQLite em um DataFrame e insere no MongoDB.

    Parâmetros:
    - consulta (list of tuples): Dados retornados do SQLite.
    - colunas (list): Lista com os nomes das colunas.
    - mongo_collection (Collection): Coleção do MongoDB onde os dados serão inseridos.
    """
    
    if not consulta:
        print("Nenhum dado para inserir no MongoDB.")
        return
    
    df = pd.DataFrame(consulta, columns=colunas)
    df = df.where(pd.notna(df), None)
    
    data_dict = df.to_dict("records")
    mongo_collection.insert_many(data_dict)
    print(data_dict)
    print("Dados inseridos com sucesso no MongoDB!")

In [96]:
consulta = resultado

client, collection = conectar_mongodb()

if collection is not None:
    enviar_para_mongodb(consulta, colunas, collection)

Conexão bem-sucedida com o MongoDB!


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [4]:
conectar_mongodb()

Conexão bem-sucedida com o MongoDB!


(MongoClient(host=['cluster001dev-shard-00-00.vgq0k.mongodb.net:27017', 'cluster001dev-shard-00-02.vgq0k.mongodb.net:27017', 'cluster001dev-shard-00-01.vgq0k.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-142osk-shard-0', tls=True),
 Collection(Database(MongoClient(host=['cluster001dev-shard-00-00.vgq0k.mongodb.net:27017', 'cluster001dev-shard-00-02.vgq0k.mongodb.net:27017', 'cluster001dev-shard-00-01.vgq0k.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-142osk-shard-0', tls=True), 'dev_mongo001_rf'), 'collection_test_mng_dev'))

In [6]:
db_sqlite = "bnc_log_igtao.db"

conn, cursor, caminho_bronze, consulta, colunas, query = conectar_bronze(db_sqlite, tabela= "tb_log_igtao")

Conectado ao SQLite Bronze: bnc_log_igtao.db
Tabelas no banco de dados:
tabela_log_igtao
tb_log_igtao


In [9]:
df_brnz = pd.read_sql_query(query, conn)
display(df_brnz)

,nome_banco,nome_tabela,nome_arquivo,tipo_arquivo,data_inclusao,dt_igtao
0,banco_logigtao,logigtao,arquivo_logigtao_20250224_104223.txt,txt,20250224142210,2025-02-24
1,banco_teste,teste,arquivo_teste_20250224_104223.txt,txt,20250224142510,2025-02-24
2,banco_teste,tabela_teste,arquivo_teste_20250224_104223.txt,txt,20250224145404,2025-02-24
3,banco_teste,tabela_teste,arquivo_teste_20250224_104223.txt,txt,20250224145507,2025-02-24
4,banco_teste,tabela_teste,arquivo_teste_20250224_104223.txt,txt,20250224145518,2025-02-24
5,banco_teste,tabela_teste,arquivo_teste_20250224_104223.txt,txt,20250224145753,2025-02-24
6,banco_mairipora,tabela_mairipora,arquivo_mairipora_20250224_104223.txt,txt,20250224150135,2025-02-24
7,banco_weather,tabela_weather,resultado_weather_20250108_163946.txt,txt,20250225184651,2025-02-25


In [3]:

file = '/workspaces/projeto_ing/apis/app/src/caldr/feriado_nacional_20250126_140402.txt' 

uri, client, collection, file = conectar_mongodb(file)

TypeError: conectar_mongodb() takes 0 positional arguments but 1 was given

In [30]:
def carregamento_landing(path=None, file=None):
    if path is None:
        path = '/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/'
        print('caminho não informado')
        print('utilizando o padrão:')
        print(f'{path}')
        
    if file is None:
        file = 'teste.txt'
    
    return path, file
    

In [29]:
carregamento_landing()

caminho não informado
utilizando o padrão:
/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/


('/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/',
 'teste.txt')

In [ ]:
## salvar no sql_conn

In [83]:
def carregamento_landing(path=None, file=None, resultado=None):
    import os
    import sys
    
    if path is None:
        path = '/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/'
        print('caminho não informado')
        print('utilizando o padrão:')
        print(f'{path}')
    
    read_landing = os.listdir(path)
    
    for reads in read_landing:
        accept = os.path.join(path, reads)
        print(f'item encontrado: {accept}')
        
        if accept.endswith('.txt'):
            resultado = pd.read_csv(accept, delimiter=';', skiprows=[0])
            # print(resultado)
        else:
            raise ValueError("Formato de arquivo não suportado")
    
    return resultado

In [77]:
carregamento_landing()

caminho não informado
utilizando o padrão:
/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_teste2_20242306_215405.txt
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_cep_mairiporã_20241014_231311.txt


('/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/',
 None,
      07601-345                           Avenida Hum      Olho D'Água  \
 0    07613-690                          Avenida Rubi      Mantiqueira   
 1    07613-655                          Avenida Ouro      Mantiqueira   
 2    07627-160                         Avenida Berna    Luiz Fagundes   
 3    07657-210                         Avenida Alpes      Mato Dentro   
 4    07613-680                        Avenida Safira      Mantiqueira   
 ..         ...                                   ...              ...   
 165  07600-598  Rodovia Prefeito Luiz Salomao Chamma          Lavapés   
 166  07605-700  Rodovia Prefeito Luiz Salomao Chamma       Rio abaixo   
 167  07600-740    Estrada Armando Barbosa de Almeida  Chácara Arantes   
 168  07604-525    Estrada Armando Barbosa de Almeida           Caceia   
 169  07605-005    Estrada Armando Barbosa de Almeida       Rio abaixo   
 
                Unnamed: 3  SP

In [84]:
path = '/workspaces/projeto_ing/tests/app/src/teste_landing/'
resultado = carregamento_landing(path)
df = pd.DataFrame(resultado)
display(df)

item encontrado: /workspaces/projeto_ing/tests/app/src/teste_landing/resultado_cep_mairiporã_20241014_231236.txt


,07601-345,Avenida Hum,Olho D'Água,Unnamed: 3,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
0,07613-690,Avenida Rubi,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
1,07613-655,Avenida Ouro,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
2,07627-160,Avenida Berna,Luiz Fagundes,(Pq Suiço Cant 1 Gl),SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
3,07657-210,Avenida Alpes,Mato Dentro,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
4,07613-680,Avenida Safira,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
...,...,...,...,...,...,...,...,...,...,...,...
162,07661-000,Rodovia Fernão Dias,Terra Preta (Terra Preta),NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
163,07663-000,Rodovia Fernão Dias,Jundiaizinho (Terra Preta),NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
164,07629-000,Rodovia Fernão Dias,Capoavinha,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
165,07600-598,Rodovia Prefeito Luiz Salomao Chamma,Lavapés,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14


In [85]:
# path = '/workspaces/projeto_ing/tests/app/src/teste_landing/'
resultado = carregamento_landing()
df = pd.DataFrame(resultado)
display(df)

caminho não informado
utilizando o padrão:
/workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_teste2_20242306_215405.txt
item encontrado: /workspaces/projeto_ing/tests/app/ambiente_test_ing/app/src/landing/resultado_cep_mairiporã_20241014_231311.txt


,07601-345,Avenida Hum,Olho D'Água,Unnamed: 3,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
0,07613-690,Avenida Rubi,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
1,07613-655,Avenida Ouro,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
2,07627-160,Avenida Berna,Luiz Fagundes,(Pq Suiço Cant 1 Gl),SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
3,07657-210,Avenida Alpes,Mato Dentro,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
4,07613-680,Avenida Safira,Mantiqueira,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
...,...,...,...,...,...,...,...,...,...,...,...
165,07600-598,Rodovia Prefeito Luiz Salomao Chamma,Lavapés,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
166,07605-700,Rodovia Prefeito Luiz Salomao Chamma,Rio abaixo,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
167,07600-740,Estrada Armando Barbosa de Almeida,Chácara Arantes,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14
168,07604-525,Estrada Armando Barbosa de Almeida,Caceia,NaN,SP,Mairiporã,São Paulo,Sudeste,11,3528502,2024-10-14


In [79]:
display(resultado)

('/workspaces/projeto_ing/tests/app/src/teste_landing/',
 None,
      07601-345                           Avenida Hum  \
 0    07613-690                          Avenida Rubi   
 1    07613-655                          Avenida Ouro   
 2    07627-160                         Avenida Berna   
 3    07657-210                         Avenida Alpes   
 4    07613-680                        Avenida Safira   
 ..         ...                                   ...   
 162  07661-000                   Rodovia Fernão Dias   
 163  07663-000                   Rodovia Fernão Dias   
 164  07629-000                   Rodovia Fernão Dias   
 165  07600-598  Rodovia Prefeito Luiz Salomao Chamma   
 166  07605-700  Rodovia Prefeito Luiz Salomao Chamma   
 
                     Olho D'Água            Unnamed: 3  SP  Mairiporã  \
 0                   Mantiqueira                   NaN  SP  Mairiporã   
 1                   Mantiqueira                   NaN  SP  Mairiporã   
 2                 Luiz Fagunde